## VAE Hawkes Process Estimation - Tutorial

In [102]:
import os

import numpy as np
import pandas as pd

from DL.mlp_model import MLPTrainer
from VARIABLES import preprocessing_var as prep
from HAWKES.hawkes import hawkes_simulations, hawkes_simulation
from HAWKES.hyperparameters import hyper_params_simulation
from HAWKES.discretisation import discretise
from DL.linear_model import linear_model
from UTILS.utils import write_csv, write_parquet, read_parquet, timer, parquet_to_csv, read_csv
from PREPROCESSING.dataset import split_data, create_datasets, create_data_loaders

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Preprocessing

In [88]:
# Training/Validation/Testing dataset generation

# Intensity Decay Parameter (β) = U(p = 1, q = 3)
# Branching Ratio (η) = U(a = 0.05, b = 0.8)
# Expected Activity (E) = 500
# Time Horizon (T) = 100
# Interval Length (∆) = 1
# Number of processes = 160_000

# Hawkes process hyper-parameters generation
params, alpha, beta, mu = hyper_params_simulation(filename="hawkes_hyperparams.parquet")

# Hawkes processes simulations
simulated_events_seqs = hawkes_simulations(alpha, beta, mu, filename='hawkes_simulations.parquet')

# Discrétiser les processus de Hawkes
discret_simulated_events_seqs = discretise(simulated_events_seqs, filename='binned_hawkes_simulations.parquet')

In [103]:
# Data Preparation

x = read_parquet("binned_hawkes_simulations.parquet")
y = read_parquet('hawkes_hyperparams.parquet')

train_x, train_y, val_x, val_y, test_x, test_y = split_data(x, y.iloc[:, [0, 2]])
train_dataset, val_dataset, test_dataset = create_datasets(train_x, train_y, val_x, val_y, test_x, test_y)
train_loader, val_loader, test_loader = create_data_loaders(train_dataset, val_dataset, test_dataset)

In [104]:
# Trained model

model, train_losses, val_losses, val_y_pred, val_eta, val_mu = MLPTrainer().train_model(train_loader, val_loader, val_x, val_y)
%load_ext tensorboard

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #                   Param %                   Kernel Shape              Mult-Adds                 Trainable
MLP                                      [128, 100]                [128, 2]                  --                             --                   --                        --                        True
├─ModuleList: 1-11                       --                        --                        (recursive)               (recursive)               --                        --                        True
│    └─Linear: 2-1                       [128, 100]                [128, 100]                10,100                     16.61%                   --                        1,292,800                 True
├─ReLU: 1-2                              [128, 100]                [128, 100]                --                             --                   --                        --              

Epoch 103/500 - train_loss: 0.3068, val_loss: 0.2918:  21%|██        | 103/500 [58:30<4:06:46, 37.30s/it]

In [86]:
# Tested model

test_y_pred, test_loss, test_eta, test_mu = MLPTrainer().test_model(test_loader, test_y)
%load_ext tensorboard

IndexError: index -9223372036854775808 is out of bounds for axis 0 with size 2

In [41]:
# α and β estimation (linear model)

param_pred, alpha_pred, beta_pred, mu_pred, eta_pred = linear_model(model, train_x, val_x, y, step_size=0.05, filename="alpha_beta_linear_estimation.parquet")

AttributeError: 'MLP' object has no attribute 'predict'

In [ ]:
# Tests and comparison (Testing Effects of Parameters = β, η, ∆, E)

# Intensity Decay Parameter (β) = U(p = 1, q = 3)
# Branching Ratio (η) = U(a = 0.2, b = 0.6)
# Expected Activity (E) = 500
# Time Horizon (T) = 100
# Interval Length (∆) = 1
# Number of processes = 200
# Number of tests = 100

# Intensity Decay Parameter (β) | Branching Ratio (η) | Interval Length (∆) | Expected Activity (E)
                                                                   
#          [0.5,2.5]                   [0.1,0.4]               0.25                   50
#         [1.75,3.75]                  [0.3,0.6]               0.5                    100
#            [3,5]                     [0.5,0.8]                1                     250
#           [0.5,3]                    [0.1,0.6]                2                     500
#           [1.5,4]                    [0.2,0.7]                5                     1000
#           [2.5,5]                    [0.3,0.8]
#           [0.5,4]                    [0.05,0.6]
#           [1.5,5]                    [0.05,0.7]
#           [0.5,5]                    [0.05,0.8]

# Hawkes process hyper-parameters generation
params, alpha, beta, mu = hyper_params_simulation(filename="hawkes_hyperparams.parquet")

# Hawkes processes simulations
simulated_events_seqs = hawkes_simulations(alpha, beta, mu, filename='hawkes_simulations.parquet')

# Discrétiser les processus de Hawkes
discret_simulated_events_seqs = discretise(simulated_events_seqs, filename='binned_hawkes_simulations.parquet')

In [82]:
import VARIABLES.evaluation_var as eval

read_parquet('2023_04_27_14_29_36_MICSHPEG8GIRARD.parquet', folder=os.path.join(eval.LOGDIRUN, 'TESTING', eval.RUN_NAME))

,eta_pred,mu_pred
0,-0.065830,0.046629
1,-0.056259,0.039393
2,-0.060766,0.043779
3,-0.070831,0.048243
4,-0.071235,0.050185
...,...,...
19995,0.074519,-0.004056
19996,-0.006365,0.098093
19997,-0.021207,0.078864
19998,0.076063,-0.043015
